<a href="https://colab.research.google.com/github/ivan74rus/DS/blob/main/forts_analyse_%D1%86%D0%B8%D0%BA%D0%BB_%D1%81%D0%BE%D0%B7%D0%B4_%D0%BE%D0%B1%D1%89%D0%B5%D0%B3%D0%BE_%D0%B4%D1%81_v2_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для автоматизации  
Сделки берем с кул скальпа   , актив уже выбран     
Создаем с pll файлы из csv   
Сохраняем списк 
Далее  циклом клеим общий файл.     
       
Сливаем их в общий файл корректно, те выкидываем лишние строки и создаем таргеты    



In [ ]:
#подключить гугл диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datetime import datetime

# 1.Лист с именами файлов для трансофрма в пикл.


In [ ]:
list_data =['si_2022_12_15.csv', 
            'deals_31_10_22.xlsx',
            'deals_01_11_22_day',
            'deals_02_11_22_day.xlsx',
            'deals_07_11_22_day.xlsx',
            'deals_08_11_22_day.xlsx',
            'deals_09_11_22_day.xlsx',
            'deals_10_11_22_day.xlsx',
            'deals_11_11_22_day.xlsx',
            'deals_14_11_22_day.xlsx',
            'deals_16_11_22_day.xlsx',
            'deals_24.01.23.xlsx',
            'deals_27.01.23.xlsx'
    
]

In [ ]:
# лист с именами пикл фалов
# в первых четырех  не оказалось цены
pickle_list =['/content/drive/MyDrive/DS/deals/pickle/si_2022_12_15.pkl', 
            #'/content/drive/MyDrive/DS/deals/pickle/deals_31_10_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_01_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_02_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_07_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_08_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_09_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_10_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_11_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_14_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_16_11_22.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_24.01.23.pkl',
            #'/content/drive/MyDrive/DS/deals/pickle/deals_27.01.23.pkl'

    
]

In [ ]:
# лист последнего дня( для тестов)
#pickle_list =['/content/drive/MyDrive/DS/deals/pickle/deals_27.01.23.pkl']

In [ ]:
# читаем и переводим в пикл
df =  pd.read_csv("/content/drive/MyDrive/DS/deals/si_2022_12_15.csv", sep=';',names=('time1','server_time','deals_number', 'price','lot','deal_type', 'nan1', 'nan2'))
df = df.drop(columns=(['deals_number','nan1', 'nan2']))

#к нижнему регистру Buy Sell
df['deal_type'] = df['deal_type'].str.lower()
#номер часа
df['hour'] = ((df['time1'] / 10000000) + 3).astype(int)
# номер минуты
df['minute'] = (df['time1'] / 100000).astype(int) - (df['time1'] / 10000000).astype(int) *100

df.to_pickle('/content/drive/MyDrive/DS/deals/pickle/si_2022_12_15.pkl')


In [ ]:
a = pd.read_pickle('/content/drive/MyDrive/DS/deals/pickle/si_2022_12_15.pkl')


In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285669 entries, 0 to 285668
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   time1        285669 non-null  int64 
 1   server_time  285669 non-null  object
 2   price        285669 non-null  int64 
 3   lot          285669 non-null  int64 
 4   deal_type    285669 non-null  object
 5   hour         285669 non-null  int64 
 6   minute       285669 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 15.3+ MB


In [ ]:
#df_temp = pd.read_pickle('/content/drive/MyDrive/DS/deals/pickle/deals_24.01.23.pkl')
#df_temp.head()



# 2.Формируем циклом из всех пиклов готовый файл с заданными свечками и таргетами. Файлы корректно сшиваются.

In [ ]:
#функциф формирования табличной свечки
# в нее передается временной кучочек дата сета 
# она готовит строку заполненоного нового дата сета
# эту строку будем добавлять в цикле


def candle (df_tmp):   
    # код падает на отсутсвующих данный во время клиры, поэтому здесь осознано
    try:

       #создаем временный пусто дф внутри функции с колонками от основного
       df_candle_row = pd.DataFrame(columns=df_candle.columns).astype(float)

       # срезы купли и продаж(чтобы питон не ругался)
       df_by = df_tmp.query('Операция == "Купля"')
       df_sell = df_tmp.query('Операция == "Продажа"')
        

       # open=0
       #df_candle_row.loc[0, 'open'] =  0
       #close из клозе выч опен, если полож, знач рост на столько, отр знач падение
       # так интуитивно понятно
       df_candle_row.loc[0, 'close'] = (df_tmp['Цена'][df_tmp.index[-1]]) - (df_tmp['Цена'][df_tmp.index[0]])  

       # хай = макс цена - опен
       df_candle_row.loc[0, 'hight'] =  df_tmp['Цена'].max() - (df_tmp['Цена'][df_tmp.index[0]])
       # лой  = мин цена - опен
       df_candle_row.loc[0, 'low'] = df_tmp['Цена'].min() - (df_tmp['Цена'][df_tmp.index[0]])

       #спреды через функц
       #df_candle_row.loc[0, 'spread_mean_by'] = spread_mean( df_by['Цена'])
       #df_candle_row.loc[0, 'spread_mean_sell'] = spread_mean( df_sell['Цена'])
         
       # различные метрики тут
       df_candle_row.loc[0, 'count_buy'] = df_by['Кол-во'].count()
       df_candle_row.loc[0, 'count_sell'] = df_sell['Кол-во'].count() 
    
       df_candle_row.loc[0, 'mean_buy'] = df_by['Кол-во'].mean()
       df_candle_row.loc[0 , 'mean_sell'] = df_sell['Кол-во'].mean()
       
       df_candle_row.loc[0, 'median_buy'] = df_by['Кол-во'].median()
       df_candle_row.loc[0, 'median_sell'] = df_sell['Кол-во'].median()
       

       # возможно лишняя инф
       df_candle_row.loc[0, 'volume_buy'] = df_by['Кол-во'].sum()
       df_candle_row.loc[0, 'volume_sell'] = df_sell['Кол-во'].sum()
      

    except IndexError:
       print (0)

    return df_candle_row



In [ ]:
#Создаем нов дата сет по свечам
features= ['close', 'hight', 'low', 'count_buy', 'count_sell', 'mean_buy','mean_sell','median_buy','median_sell', 'volume_buy' ,'volume_sell', 'target']
df_candle = pd.DataFrame(columns=features).astype(float)


In [ ]:
#Списки активов,чтобы не править руками постоянно при сменах контрактов
si = ['SiZ2' ,'SiH3']
ri = ['RIZ2' ,'RIH3']
sr = ['SRZ2','SRH3']
br = ['BRG3']
#выбираем аткив
aktiv_list = ri


In [ ]:
#сборный дата сет

df_candle_all = pd.DataFrame(columns=features)
for pickle_df in pickle_list:
  # обновляем дата сет в ячейке
  #df_candle = pd.DataFrame(columns=features)
  #обрабатываем датасет
  df = pd.read_pickle(pickle_df)
  #выбираем актив
  df = df.query(' @aktiv_list in instrument')
  #преобразование типа в формат времени
  df['Время'] = df['Время'].astype("datetime64[ns]")
  #добавляем колонку с номером минуты
  df['minute'] = df['Время'].dt.minute
  df['hour'] = df ['Время'].dt.hour

  #задаем условия  для цикла
  start_hour = df['hour'].min()
  close_hour = df['hour'].max()
  #КОНТРОЛЬ исходного файла
  print ('имя дата сета', pickle_df)
  print ('df.shape', df.shape)

  #цикл по заполнению , вызываем в нем функцию
  #обновляем датасет  
  df_candle = pd.DataFrame(columns=features)
  #>>>>>>>>>>
  #ТУТ ЗАДАЕМ ВРЕМЕННОЙ ИНТРЕВАЛ СВЕЧКИ
  for i in range (start_hour, close_hour+1):
    for j in range (0,60,5):
       # 5 ,15 min ( не забывай менять range)
       df_tmp = df[(df['hour'] == i)  & ((df['minute'] >= j) & (df['minute'] < 5+j ) ) ]
       
       #минутный ( не забывай менять range)
       #df_tmp = df [(df['hour'] == i ) & (df['minute'] == j)]  
       
       #применяем функцию
       df_candle = df_candle.append(candle(df_tmp), ignore_index=True)
      
       #контроль под цикла цикла
       #print ('hour=',i, 'minute=', j)
  #>>>>>>>>
  #УСЛОВИЕ таргет класификатор на три 0- додж, падение и рост тела свечи
  # тело свечи белое (close>0) &  модуль тело свечи  больше  УСЛОВИя   (candle_len)
  candle_len = candle_median = df_candle['close'].abs().median() *2
  df_candle.loc[((df_candle['close'] >= 0) & (df_candle['close'].abs() >= candle_len) ),'target'] = 1
  df_candle.loc[((df_candle['close'] <= 0) & (df_candle['close'].abs() >= candle_len) ),'target'] = -1
  df_candle.loc[df_candle['target'].isna(), 'target'] = 0

  #смещение таргета   подниматем на одну , тем самым будущее в настоящее перенос
  df_candle['target'] = df_candle['target'].shift(-1)
  # дроп последней строки, которая  никого не предскажет
  df_candle =  df_candle.drop(index=len(df_candle)-1)
  
  #КОНТРОЛЬ размера полученного файла
  print ('df_candle.shape',df_candle.shape)
  #сшиваем дата сеты
  df_candle_all = pd.concat([df_candle_all, df_candle], ignore_index=True)
print ('df_candle_all.shape = ',df_candle_all.shape)
#переименуем обратно
df_candle = df_candle_all

имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_07_11_22.pkl
df.shape (75173, 9)
0
0
0
0
0
0
0
df_candle.shape (136, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_08_11_22.pkl
df.shape (66770, 9)
0
0
0
df_candle.shape (116, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_09_11_22.pkl
df.shape (87692, 9)
0
0
0
0
0
0
0
0
0
0
0
0
0
df_candle.shape (142, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_10_11_22.pkl
df.shape (85537, 9)
0
0
0
0
0
0
0
df_candle.shape (124, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_11_11_22.pkl
df.shape (76220, 9)
0
0
0
0
0
0
df_candle.shape (173, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_14_11_22.pkl
df.shape (60551, 9)
0
0
0
df_candle.shape (116, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_16_11_22.pkl
df.shape (74289, 9)
0
0
0
0
0
0
0
0
0
0
0
0
0
df_candle.shape (130, 12)
имя дата сета /content/drive/MyDrive/DS/deals/pickle/deals_24.01.23.pk

In [ ]:
df_candle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        1074 non-null   float64
 1   hight        1074 non-null   float64
 2   low          1074 non-null   float64
 3   count_buy    1074 non-null   float64
 4   count_sell   1074 non-null   float64
 5   mean_buy     1074 non-null   float64
 6   mean_sell    1074 non-null   float64
 7   median_buy   1074 non-null   float64
 8   median_sell  1074 non-null   float64
 9   volume_buy   1074 non-null   float64
 10  volume_sell  1074 non-null   float64
 11  target       1074 non-null   float64
dtypes: float64(12)
memory usage: 100.8 KB


In [ ]:
#баланс классов
df_candle['target'].value_counts(normalize=True)

 0.0    0.747672
 1.0    0.137803
-1.0    0.114525
Name: target, dtype: float64

In [ ]:
#переименовываем
#df_candle_si = df_candle
#сохраняем
df_candle.to_pickle('/content/drive/MyDrive/DS/deals/pickle/ri_5min_until_27.01.23.pkl')


#ML